In [3]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "hotel-reviews"

In [4]:
library(ggplot2)
library(tibble)
library(gridExtra)
library(tm)
library(SnowballC)
library(syuzhet)
library(plotly)
library(htmlwidgets)
library('IRdisplay')


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: NLP


Attaching package: ‘NLP’


The following object is masked from ‘package:ggplot2’:

    annotate


The following object is masked from ‘package:httr’:

    content



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout


The following object is masked from ‘package:httr’:

    config




In [5]:
hotel_reviews = read_csv("/kaggle/input/hotel-reviews/Datafiniti_Hotel_Reviews.csv")
hotel_general_info = read_csv("/kaggle/input/hotel-reviews/7282_1.csv")
hotel_reviews_Jun19 = read_csv("/kaggle/input/hotel-reviews/Datafiniti_Hotel_Reviews_Jun19.csv")

Rows: 10000 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (19): id, address, categories, primaryCategories, city, country, keys, ...
dbl   (3): latitude, longitude, reviews.rating
dttm  (3): dateAdded, dateUpdated, reviews.date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 35912 Columns: 19
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (12): address, categories, city, country, name, postalCode, province, r...
dbl   (3): latitude, longitude, reviews.rating
lgl   (2): reviews.doRecommend, reviews.id
dttm  (2): reviews.date, reviews.dateAdded

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 10000 Columns: 26
── Column specification ──────────────

In [6]:
glimpse(hotel_reviews)
glimpse(hotel_reviews_Jun19)

Rows: 10,000
Columns: 25
$ id                   <chr> "AVwc252WIN2L1WUfpqLP", "AVwc252WIN2L1WUfpqLP", "…
$ dateAdded            <dttm> 2016-10-30 21:42:42, 2016-10-30 21:42:42, 2016-1…
$ dateUpdated          <dttm> 2018-09-10 21:06:27, 2018-09-10 21:06:27, 2018-0…
$ address              <chr> "5921 Valencia Cir", "5921 Valencia Cir", "5921 V…
$ categories           <chr> "Hotels,Hotels and motels,Hotel and motel reserva…
$ primaryCategories    <chr> "Accommodation & Food Services", "Accommodation &…
$ city                 <chr> "Rancho Santa Fe", "Rancho Santa Fe", "Rancho San…
$ country              <chr> "US", "US", "US", "US", "US", "US", "US", "US", "…
$ keys                 <chr> "us/ca/ranchosantafe/5921valenciacir/359754519", …
$ latitude             <dbl> 32.99096, 32.99096, 32.99096, 39.15593, 39.15593,…
$ longitude            <dbl> -117.18614, -117.18614, -117.18614, -76.71634, -7…
$ name                 <chr> "Rancho Valencia Resort Spa", "Rancho Valencia Re…
$ postalCode   

In [7]:
colnames(hotel_reviews_Jun19)

unique(hotel_reviews_Jun19$reviews.dateAdded)

[1] "id"                   "dateAdded"            "dateUpdated"         
 [4] "address"              "categories"           "primaryCategories"   
 [7] "city"                 "country"              "keys"                
[10] "latitude"             "longitude"            "name"                
[13] "postalCode"           "province"             "reviews.date"        
[16] "reviews.dateAdded"    "reviews.dateSeen"     "reviews.rating"      
[19] "reviews.sourceURLs"   "reviews.text"         "reviews.title"       
[22] "reviews.userCity"     "reviews.userProvince" "reviews.username"    
[25] "sourceURLs"           "websites"

[1] NA

Column names are not good, need to change as well as clean the data.

# Data Cleaning

In [8]:
hotel_reviews <- hotel_reviews %>%
rename(reviews_date = reviews.date,
       reviews_dateSeen = reviews.dateSeen,
       reviews_rating = reviews.rating,
       reviews_sourceURLs = reviews.sourceURLs,
       reviews_text = reviews.text,
       reviews_title = reviews.title,
       reviews_userCity = reviews.userCity,
       reviews_userProvince = reviews.userProvince,
       reviews_username = reviews.username)

In [9]:
# Same for hotel_reviews_Jun19

hotel_reviews_Jun19 <- hotel_reviews_Jun19 %>%
rename(reviews_date = reviews.date,
       reviews_dateAdded = reviews.dateAdded,
       reviews_dateSeen = reviews.dateSeen,
       reviews_rating = reviews.rating,
       reviews_sourceURLs = reviews.sourceURLs,
       reviews_text = reviews.text,
       reviews_title = reviews.title,
       reviews_userCity = reviews.userCity,
       reviews_userProvince = reviews.userProvince,
       reviews_username = reviews.username)

In [10]:
unique(hotel_reviews_Jun19$reviews_dateAdded)

[1] NA

In [11]:
# Then drop the empty column, reviews_dateAdded

hotel_reviews_Jun19_1 = subset(hotel_reviews_Jun19, select = -c(reviews_dateAdded))

In [12]:
colnames(hotel_reviews_Jun19_1)

[1] "id"                   "dateAdded"            "dateUpdated"         
 [4] "address"              "categories"           "primaryCategories"   
 [7] "city"                 "country"              "keys"                
[10] "latitude"             "longitude"            "name"                
[13] "postalCode"           "province"             "reviews_date"        
[16] "reviews_dateSeen"     "reviews_rating"       "reviews_sourceURLs"  
[19] "reviews_text"         "reviews_title"        "reviews_userCity"    
[22] "reviews_userProvince" "reviews_username"     "sourceURLs"          
[25] "websites"

In [13]:
# Merge both hotel_reviews and hotel_reviews_Jun19_1

hotel_reviews_full <- rbind(hotel_reviews, hotel_reviews_Jun19_1)

In [14]:
#hotel_reviews_full

In [15]:
# After merging, need to check to see if there are empty values

# Checking missing values

missing_values <- function(df) {
    missing_val <- c()
    for (c in colnames(df)) {
        missing <- sum(is.na(df[[c]]))
        missing_val <- c(missing_val, missing)
    }
    tibble(colnames(df), missing_val)
}

missing_values(hotel_reviews_full)

colnames(df),missing_val
<chr>,<int>
id,0
dateAdded,0
dateUpdated,0
address,0
categories,0
primaryCategories,0
city,0
country,0
keys,0


In [16]:
unique(hotel_reviews_full$province)

[1] "CA" "MD" "WA" "MO" "PA" "GA" "NJ" "IL" "FL" "MA" "LA" "NV" "NY" "HI" "TX"
[16] "AZ" "NH" "MI" "VA" "NC" "CO" "KY" "KS" "DE" "ID" "RI" "IA" "ME" "OH" "IN"
[31] "OR" "AK" "AR" "TN" "SD" "WV" "WI" "VT" "NM" "SC" "MT" "MN" "CT" "UT" "NE"
[46] "OK" "WY" "ND" "MS" "AL"

In [17]:
# All provinces are US States, change it to states as well as postal code to zipcode

hotel_reviews_full <- hotel_reviews_full %>%
rename(zipcode = postalCode,
       state = province)

In [18]:
colnames(hotel_reviews_full)

[1] "id"                   "dateAdded"            "dateUpdated"         
 [4] "address"              "categories"           "primaryCategories"   
 [7] "city"                 "country"              "keys"                
[10] "latitude"             "longitude"            "name"                
[13] "zipcode"              "state"                "reviews_date"        
[16] "reviews_dateSeen"     "reviews_rating"       "reviews_sourceURLs"  
[19] "reviews_text"         "reviews_title"        "reviews_userCity"    
[22] "reviews_userProvince" "reviews_username"     "sourceURLs"          
[25] "websites"

In [19]:
# There are unnecesary data (columns) for us to analyze
# Let's drop them to make it clear

hotel_reviews_full_clean = subset(hotel_reviews_full, select = -c(categories, keys, reviews_dateSeen, reviews_sourceURLs, reviews_userCity, reviews_userProvince))


In [20]:
glimpse(hotel_reviews_full_clean)

Rows: 20,000
Columns: 19
$ id                <chr> "AVwc252WIN2L1WUfpqLP", "AVwc252WIN2L1WUfpqLP", "AVw…
$ dateAdded         <dttm> 2016-10-30 21:42:42, 2016-10-30 21:42:42, 2016-10-3…
$ dateUpdated       <dttm> 2018-09-10 21:06:27, 2018-09-10 21:06:27, 2018-09-1…
$ address           <chr> "5921 Valencia Cir", "5921 Valencia Cir", "5921 Vale…
$ primaryCategories <chr> "Accommodation & Food Services", "Accommodation & Fo…
$ city              <chr> "Rancho Santa Fe", "Rancho Santa Fe", "Rancho Santa …
$ country           <chr> "US", "US", "US", "US", "US", "US", "US", "US", "US"…
$ latitude          <dbl> 32.99096, 32.99096, 32.99096, 39.15593, 39.15593, 39…
$ longitude         <dbl> -117.18614, -117.18614, -117.18614, -76.71634, -76.7…
$ name              <chr> "Rancho Valencia Resort Spa", "Rancho Valencia Resor…
$ zipcode           <chr> "92067", "92067", "92067", "21076", "21076", "21076"…
$ state             <chr> "CA", "CA", "CA", "MD", "MD", "MD", "MD", "MD", "MD"…
$ reviews_date 

In [21]:
# We can simplify sourceURLs as review_sources

hotel_reviews_full_clean <- hotel_reviews_full_clean %>%
mutate(review_sources = gsub("^https?://(www\\.)?([^/]+).*", "\\2", hotel_reviews_full_clean$sourceURLs))

In [22]:
# Also, let's make full address

hotel_reviews_full_clean <- hotel_reviews_full_clean %>%
mutate(full_address_1 = paste(hotel_reviews_full_clean$address, 
                            hotel_reviews_full_clean$city, 
                            hotel_reviews_full_clean$state, sep = ", ")) %>%
mutate(full_address_2 = paste(full_address_1, hotel_reviews_full_clean$zipcode, sep = " ")) %>%
mutate(full_address = paste(full_address_2, "United States", sep = ", "))

In [23]:
unique(hotel_reviews_full_clean$review_sources)

[1] "hotels.com"                   "yellowbook.com"              
 [3] "yellowpages.com"              "gayot.com"                   
 [5] "foursquare.com"               "tripadvisor.com"             
 [7] "expedia.com"                  "booking.com"                 
 [9] "telegraph.co.uk"              "superpages.com"              
[11] "citysearch.com"               "marriott.ugc.bazaarvoice.com"
[13] "zoover.nl"                    "insiderpages.com"            
[15] "kudzu.com"                    "weddingwire.com"             
[17] "bbb.org"                      "trip.com"                    
[19] "local.com"                    "marriott.com"                
[21] "tripadvisor.ca"               "timeout.com"                 
[23] "travelandleisure.com"         "dexknows.com"                
[25] "onesouthlumina.info"          "belairecovemotel.com"        
[27] "tripadvisor.co.uk"

In [24]:
unique(hotel_reviews_full_clean$full_address)

[1] "5921 Valencia Cir, Rancho Santa Fe, CA 92067, United States"                   
   [2] "7520 Teague Rd, Hanover, MD 21076, United States"                              
   [3] "315 SE Olympia Dr, Vancouver, WA 98684, United States"                         
   [4] "106 W 12th St, Kansas City, MO 64105, United States"                           
   [5] "10611 Standing Stone Rd, Huntingdon, PA 16652, United States"                  
   [6] "102 Valley Dr, Perry, GA 31069, United States"                                 
   [7] "12 4th St, San Francisco, CA 94103, United States"                             
   [8] "998 W Landis Ave, Vineland, NJ 8360, United States"                            
   [9] "250 Waterfront St, Oxon Hill, MD 20745, United States"                         
  [10] "5300 Grand del Mar Ct, San Diego, CA 92130, United States"                     
  [11] "10300 W Higgins Rd, Des Plaines, IL 60018, United States"                      
  [12] "2261 E Irlo Bronson Memorial Hwy, Kissimmee, FL 34744, United States"          
  [13] "203 N Wabash Ave, Chicago, IL 60601, United States"                            
  [14] "107 Merrimac Street, Boston, MA 2114, United States"                           
  [15] "535 Tchoupitoulas St, New Orleans, LA 70130, United States"                    
  [16] "200 North Main Street, Las Vegas, NV 89101, United States"                     
  [17] "200 Fremont Street, Las Vegas, NV 89101, United States"                        
  [18] "655 Boylston Street, Boston, MA 2116, United States"                           
  [19] "225 Albany Street, Boston, MA 2118, United States"                             
  [20] "801 Collins Avenue, Miami Beach, FL 33139, United States"                      
  [21] "233 West 49th Street, New York, NY 10019, United States"                       
  [22] "129 Paoakalani Avenue, Honolulu, HI 96815, United States"                      
  [23] "27 Barclay St, New York, NY 10007, United States"                              
  [24] "22 East 29th Street, New York, NY 10016, United States"                        
  [25] "210 O'Keefe Ave, New Orleans, LA 70112, United States"                         
  [26] "320 Decatur St, New Orleans, LA 70130, United States"                          
  [27] "350 Ocean Drive, Miami Beach, FL 33139, United States"                         
  [28] "160 East Flamingo Road, Las Vegas, NV 89109, United States"                    
  [29] "431 Broome St, New York, NY 10013, United States"                              
  [30] "100 Stuart Street, Boston, MA 2116, United States"                             
  [31] "500 Brickell Key Drive, Miami, FL 33131, United States"                        
  [32] "10 Avery Street, Boston, MA 2111, United States"                               
  [33] "Two East 55th Street at Fifth Avenue, New York, NY 10022, United States"       
  [34] "328 East 1st Street, Los Angeles, CA 90012, United States"                     
  [35] "630 North Rush Street, Chicago, IL 60611, United States"                       
  [36] "311 North Morgan Street, Chicago, IL 60611, United States"                     
  [37] "660 North State Street, Chicago, IL 60654, United States"                      
  [38] "20 West Kinzie Street, Chicago, IL 60654, United States"                       
  [39] "530 N LaSalle Dr, Chicago, IL 60654, United States"                            
  [40] "225 North Canon Drive, Beverly Hills, CA 90210, United States"                 
  [41] "55 5th Street, San Francisco, CA 94103, United States"                         
  [42] "1060 Adella Ave, Coronado, CA 92118, United States"                            
  [43] "1200 Arch St, Philadelphia, PA 19107, United States"                           
  [44] "204 W 23rd St, New York, NY 10011, United States"                              
  [45] "111 W Adams St, Chicago, IL 60603, United States"                              
  [46] "99 Convention Center Dr, Las Vegas,

In [25]:
hotel_reviews_full_clean = subset(hotel_reviews_full_clean, select = -c(sourceURLs, full_address_1, full_address_2))

In [26]:
# Before visualization, let's check empty values 1 more time

missing_values(hotel_reviews_full_clean)

colnames(df),missing_val
<chr>,<int>
id,0
dateAdded,0
dateUpdated,0
address,0
primaryCategories,0
city,0
country,0
latitude,0
longitude,0


In [27]:
# hotel_reviews_full_clean

In [28]:
# Let's analyze user reviews
# First, create corpus

corpus <- iconv(hotel_reviews_full_clean$reviews_text, to = "UTF-8", sub = "byte")
corpus <- Corpus(VectorSource(corpus))

cleaned_corpus <- tm_map(corpus, content_transformer(tolower))
cleaned_corpus <- tm_map(cleaned_corpus, removePunctuation)
cleaned_corpus <- tm_map(cleaned_corpus, removeNumbers)
cleaned_corpus <- tm_map(cleaned_corpus, removeWords, stopwords('english'))
cleaned_corpus <- tm_map(cleaned_corpus, stripWhitespace)
cleaned_corpus <- tm_map(cleaned_corpus, stemDocument)

class(cleaned_corpus)

# hotel_reviews_full_clean <- hotel_reviews_full_clean %>%
# mutate(review_sentences = get_sentences(reviews_text))

# hotel_reviews_full_clean <- hotel_reviews_full_clean %>%
# mutate(sentimentr_reviews = sentiment(hotel_reviews_full_clean$reviews_text))

Warning message in tm_map.SimpleCorpus(corpus, content_transformer(tolower)):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(cleaned_corpus, removePunctuation):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(cleaned_corpus, removeNumbers):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(cleaned_corpus, removeWords, stopwords("english")):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(cleaned_corpus, stripWhitespace):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(cleaned_corpus, stemDocument):
“transformation drops documents”


[1] "SimpleCorpus" "Corpus"

In [29]:
# Clean and utilize Syuzhet
cleaned_texts <- sapply(cleaned_corpus, as.character)

# Run sentiment analysis
sentiment_scores <- get_sentiment(cleaned_texts, method = "syuzhet")

# Attach results to your dataframe
hotel_reviews_full_clean$sentiment_score_syuzhet <- sentiment_scores

In [30]:
length(hotel_reviews_full_clean$sentiment_score_syuzhet)

[1] 20000

In [31]:
hotel_reviews_full_clean <- hotel_reviews_full_clean %>%
  mutate(sentiment_label_syuzhet = case_when(
    sentiment_score_syuzhet > 0  ~ "positive",
    sentiment_score_syuzhet < 0  ~ "negative",
    TRUE                 ~ "neutral"
  ))

In [32]:
hotel_reviews_full_clean

id,dateAdded,dateUpdated,address,primaryCategories,city,country,latitude,longitude,name,⋯,reviews_date,reviews_rating,reviews_text,reviews_title,reviews_username,websites,review_sources,full_address,sentiment_score_syuzhet,sentiment_label_syuzhet
<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,⋯,<dttm>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
AVwc252WIN2L1WUfpqLP,2016-10-30 21:42:42,2018-09-10 21:06:27,5921 Valencia Cir,Accommodation & Food Services,Rancho Santa Fe,US,32.99096,-117.18614,Rancho Valencia Resort Spa,⋯,2013-11-14,5,Our experience at Rancho Valencia was absolutely perfect from beginning to end!!!! We felt special and very happy during our stayed. I would come back in a heart beat!!!,Best romantic vacation ever!!!!,Paula,http://www.ranchovalencia.com,hotels.com,"5921 Valencia Cir, Rancho Santa Fe, CA 92067, United States",1.55,positive
AVwc252WIN2L1WUfpqLP,2016-10-30 21:42:42,2018-09-10 21:06:27,5921 Valencia Cir,Accommodation & Food Services,Rancho Santa Fe,US,32.99096,-117.18614,Rancho Valencia Resort Spa,⋯,2014-07-06,5,Amazing place. Everyone was extremely warm and welcoming. We've stayed at some top notch places and this is definitely in our top 2. Great for a romantic getaway or take the kids along as we did. Had a couple stuffed animals waiting for our girls upon arrival. Can't wait to go back.,Sweet sweet serenity,D,http://www.ranchovalencia.com,hotels.com,"5921 Valencia Cir, Rancho Santa Fe, CA 92067, United States",0.75,positive
AVwc252WIN2L1WUfpqLP,2016-10-30 21:42:42,2018-09-10 21:06:27,5921 Valencia Cir,Accommodation & Food Services,Rancho Santa Fe,US,32.99096,-117.18614,Rancho Valencia Resort Spa,⋯,2015-01-02,5,"We booked a 3 night stay at Rancho Valencia to play some tennis, since it is one of the highest rated tennis resorts in America. This place is really over the top from a luxury standpoint and overall experience. The villas are really perfect, the staff is great, attention to details (includes fresh squeezed orange juice each morning), restaurants, bar and room service amazing, and the tennis program was really impressive as well. We will want to come back here again.",Amazing Property and Experience,Ron,http://www.ranchovalencia.com,hotels.com,"5921 Valencia Cir, Rancho Santa Fe, CA 92067, United States",3.30,positive
AVwdOclqIN2L1WUfti38,2015-11-28 19:19:35,2018-09-10 21:06:16,7520 Teague Rd,Accommodation & Food Services,Hanover,US,39.15593,-76.71634,Aloft Arundel Mills,⋯,2016-05-15,2,Currently in bed writing this for the past hr 1/2 there have been dogs barking and squealing call the front desk to advise basically to be told there's nothing they can do. 315.00 and I can't sleep.,"Never again...beware, if you want sleep.",jaeem2016,"http://www.starwoodhotels.com/alofthotels/property/overview/index.html?propertyID=3050?scid=45f93f1b-bd77-45c9-8dab-83b6a417f6fe,http://www.aloftarundelmills.com,https://www.marriott.com/hotels/travel/bwima-aloft-arundel-mills-bwi-airport/",yellowbook.com,"7520 Teague Rd, Hanover, MD 21076, United States",-0.25,negative
AVwdOclqIN2L1WUfti38,2015-11-28 19:19:35,2018-09-10 21:06:16,7520 Teague Rd,Accommodation & Food Services,Hanover,US,39.15593,-76.71634,Aloft Arundel Mills,⋯,2016-07-09,5,I live in Md and the Aloft is my Home away from home...we stayed 1 night 7-7-16 ...Staff is great ! Especially Olivia who was Extra special because she remembered me by my voice over the phone ...which tells me she is very alert and pays attention to the customer their needs.AND SHE DID ! Thumbs up... More,ALWAYS GREAT STAY...,MamaNiaOne,"http://www.starwoodhotels.com/alofthotels/property/overview/index.html?propertyID=3050?scid=45f93f1b-bd77-45c9-8dab-83b6a417f6fe,http://www.aloftarundelmills.com,https://www.marriott.com/hotels/travel/bwima-aloft-arundel-mills-bwi-airport/",yellowbook.com,"7520 Teague Rd, Hanover, MD 21076, United States",1.35,positive
AVwdOclqIN2L1WUfti38,2015-11-28 19:19:35,2018-09-10 21:06:16,7520 Teague Rd,Accommodation & Food Servic

In [33]:
missing_values(hotel_reviews_full_clean)

colnames(df),missing_val
<chr>,<int>
id,0
dateAdded,0
dateUpdated,0
address,0
primaryCategories,0
city,0
country,0
latitude,0
longitude,0


In [34]:
# Import CSV for Power BI

# write.csv(hotel_reviews_full_clean, "hotel_reviews_syuzhet_sentiment.csv", row.names = FALSE)

Data cleaning is over, let's jump onto data visualization!

# Data Visualization

In [61]:
# First, let's check review dates

review_dates <- hotel_reviews_full_clean$reviews_date

oldest_date <- min(review_dates, na.rm = TRUE)

latest_date <- max(review_dates, na.rm = TRUE)

oldest_date
latest_date

[1] "2002-07-24 UTC"

[1] "2019-01-30 UTC"

In [35]:
# Top 10 States by Number of Hotels

hotels_states <- hotel_reviews_full_clean %>%
group_by(state) %>%
summarise(total_hotels = n()) %>%
arrange(desc(total_hotels)) %>%
head(10)

top_10_states <- plot_ly(x = hotels_states$total_hotels,
                         y = reorder(hotels_states$state, hotels_states$total_hotels),
                         type = "bar",
                         text = paste("State:", hotels_states$state, 
                                      "<br>Number of Hotels:", hotels_states$total_hotels),
                         width = 1,
                         orientation = "h"
                        ) %>%
layout(title = "Top 10 States by Number of Hotels",
       xaxis = list(title = "Number of Hotels"),
       yaxis = list(title = "State"),
       bargap = 0.2)


saveWidget(top_10_states, "top_10_states.html")
display_html('<iframe src="top_10_states.html" width=80% height=450></iframe>')

As we can see, **California** has the highest number of hotels nationwide.

In [36]:
# Top 10 States by Hospitality (Based on Avg. Ratings) Nationwide

hotel_ratings_state <- hotel_reviews_full_clean %>%
group_by(state) %>%
summarise(avg_rating = mean(reviews_rating)) %>%
arrange(desc(avg_rating)) %>%
head(10)

# hotel_ratings

top_10_states_hospitality <- plot_ly(x = hotel_ratings_state$avg_rating,
                         y = reorder(hotel_ratings_state$state, hotel_ratings_state$avg_rating),
                         type = "bar",
                         text = paste("State:", hotel_ratings_state$state, 
                                      "<br>Hospitality Rating:", hotel_ratings_state$avg_rating),
                         width = 1,
                         orientation = "h"
                        ) %>%
layout(title = "Top 10 States by Hospitality (Based on Avg. Ratings) Nationwide",
       xaxis = list(title = "Hospitality Rating"),
       yaxis = list(title = "State"),
       bargap = 0.2)


saveWidget(top_10_states_hospitality, "top_10_states_hospitality.html")
display_html('<iframe src="top_10_states_hospitality.html" width=80% height=450></iframe>')

Apparently, **Utah** is top state with the hospital rating.

In [37]:
# Top 10 Hotels by Number of Positive Reviews Based on Syuzhet

hotel_ratings_syuzhet <- hotel_reviews_full_clean %>%
group_by(name) %>%
summarise(total_positive_reviews_syuzhet = sum(sentiment_label_syuzhet == 'positive'),
          total_reviews_syuzhet = n(),
          positive_rating_percentage = 100*(sum(sentiment_label_syuzhet == 'positive')/n())) %>%
arrange(desc(total_positive_reviews_syuzhet)) %>%
head(10)

# hotel_ratings_syuzhet

hotel_ratings_syuzhet_fig <- plot_ly(
  data = hotel_ratings_syuzhet,
  y = ~name,
  x = ~total_positive_reviews_syuzhet,
  type = "bar",
  name = "Positive Reviews"
) %>%
  add_trace(
    x = ~positive_rating_percentage,
    name = "Positive Review Percentage"
  ) %>%
  layout(
    title = "Top 10 Hotels by Positive Reviews Nationwide",
    barmode = "stack",
    xaxis = list(title = "Positive Reviews / Positive Review %"),
    yaxis = list(title = "Hotel", categoryorder = "total ascending")
  )

# hotel_ratings_syuzhet_fig

# Save and display
saveWidget(hotel_ratings_syuzhet_fig, "hotel_ratings_syuzhet_fig.html")
display_html('<iframe src="hotel_ratings_syuzhet_fig.html" width=100% height=450></iframe>')

The reason I chose to measure performance by the number of positive reviews rather than the average rating is that it provides a more accurate reflection of overall guest satisfaction. For instance, consider a hotel with just one 5-star review versus another with thirty 5-star reviews and ten 4-star reviews — the latter clearly offers a more reliable indication of consistent quality and guest experience.

In [55]:
# Lastly, which platform is popular for hotel reviews?

hotel_ratings_platform <- hotel_reviews_full_clean %>%
group_by(review_sources) %>%
summarise(total_reviews_per_source = n()) %>%
mutate(platform_percentage = 100*(total_reviews_per_source/nrow(hotel_reviews_full_clean)))

# hotel_ratings_platform

# Which platform takes a large portion?

hotel_ratings_platform_fig <- plot_ly(hotel_ratings_platform, labels = ~review_sources, values = ~platform_percentage, type = 'pie',
        textposition = 'inside',
        textinfo = 'label+percent',
        insidetextfont = list(color = '#FFFFFF'),
        hoverinfo = 'text',
        text = ~paste('Platform Name:', review_sources, '<br>Platform Percentage:', platform_percentage, "%"),
        marker = list(colors = colors,
                      line = list(color = '#FFFFFF', width = 1)),
                      #The 'pull' attribute can also be used to create space between the sectors
        showlegend = FALSE)
hotel_ratings_platform_fig <- hotel_ratings_platform_fig %>% layout(title = 'Which Platform is Commonly Used for Hotel Reviews?',
         xaxis = list(showgrid = FALSE, zeroline = FALSE, showticklabels = FALSE),
         yaxis = list(showgrid = FALSE, zeroline = FALSE, showticklabels = FALSE))

# Save and display
saveWidget(hotel_ratings_platform_fig, "hotel_ratings_platform_fig.html")
display_html('<iframe src="hotel_ratings_platform_fig.html" width=100% height=450></iframe>')

# Conclusion

**To summarize:**

From "2002-07-24 UTC" to "2019-01-30 UTC":

1. California has the highest number of hotels.
2. Utah has the highest hospitality rating among 50 states.
3. Hyatt House Seattle/Downtown is the highest rated hotels nationwide.
4. Lastly, tripadvisor.com is the most popular source, as it takes 48.4% of ratings.